In [2]:
from collections import defaultdict
from functools import cache
import pandas as pd
import pyodbc
import requests
import heapq
import numpy as np
from scipy.spatial import distance
import torch
from torch.utils.data import Dataset, DataLoader
from math import floor
from torch import nn

Fetch rows from the database. Run once and only when needed. 

Load in the data in a dataframe.

In [ ]:
df: pd.DataFrame = pd.read_pickle('bikeshare_data.pkl')
print(df.shape[0])

8240287


Next, we fetch station_information, which will be used in later functions

In [4]:
BIKE_BASE_ENDPOINT = "https://tor.publicbikesystem.net/ube/gbfs/v1/en/" 
ENDPOINTS = {
        "station_status": BIKE_BASE_ENDPOINT + "station_status",
        "station_information": BIKE_BASE_ENDPOINT + "station_information",
}


def _fetch_station_status(endpoints: dict[str, str]) -> list[dict]:
    """Return a sorted list (by station_id) containing station status for all stations."""
    response = requests.get(endpoints["station_status"])
    response_data: dict = response.json()
    return response_data["data"]["stations"]

def _fetch_station_information(endpoints: dict[str, str]) -> list[dict]:
    """Return a sorted list (by station_id) containing station information for all stations."""
    response = requests.get(endpoints["station_information"])
    response_data: dict = response.json()
    return response_data["data"]["stations"]

station_status = _fetch_station_status(ENDPOINTS)
station_information = _fetch_station_information(ENDPOINTS)


Load the working station IDS, discarding EOL stations and other define ID related functions

In [5]:
def get_all_IDs() -> list[str]:
    """Return a sorted list with all the stationIDs."""
    ids = []
    status_i = 0
    information_i = 0
    while status_i < len(station_status) and information_i < len(station_information):
        status_station_id = int(station_status[status_i]["station_id"])
        information_station_id = int(station_information[information_i]["station_id"])
        if status_station_id < information_station_id:
            status_i += 1
        elif information_station_id < status_station_id:
            information_i += 1
        else:
            ids.append(station_status[status_i]["station_id"])
            status_i += 1
            information_i += 1
    
    return ids

original_station_ids = get_all_IDs()
NUM_STATIONS_ORIGINAL = len(original_station_ids)


# to be used for testing
def id_to_original_ID(id: int) -> int:
    """Convert the id (internal) to the original ID in the data."""
    return original_station_ids[id]

@cache
def station_encoding(station_id: int) -> list[int]:
    """Return a one hot encoding of the original station ID, station_id, based on its index in original_station_ids."""
    one_hot_encoding = [0] * NUM_STATIONS_ORIGINAL
    index = original_station_ids.index(station_id)
    one_hot_encoding[index] = 1
    return tuple(one_hot_encoding)  


Weather related code

In [6]:
# list of all weather status, taking from the open weather map website
WEATHER_STATUSES = [
        200, 201, 202, 210, 211, 212, 221, 230, 231, 232,
        300, 301, 302, 310, 311, 312, 313, 314, 321,
        500, 501, 502, 503, 504, 511, 520, 521, 522, 531,
        600, 601, 602, 611, 612, 613, 615, 616, 620, 621, 622,
        701, 711, 721, 731, 741, 751, 761, 762, 771, 781,
        800,
        801, 802, 803, 804
]
NUM_STATUSES =  len(WEATHER_STATUSES)

@cache
def weather_encoding(weather_status: int) -> tuple[int]:
    """Return a one hot encoding of weather_status based on its index in WEATHER_STATUSES."""
    one_hot_encoding = [0] * NUM_STATUSES
    index = WEATHER_STATUSES.index(weather_status)
    one_hot_encoding[index] = 1
    return tuple(one_hot_encoding)



In [ ]:
# Certain stations are turned of for a day or some period of time, we ignore those stations in our analysis

group_sizes = df.groupby("StationID").size()
size_to_groups: dict = group_sizes.groupby(group_sizes).apply(lambda x: list(x.index)).to_dict()

station_ids = size_to_groups[max(size_to_groups.keys())]
NUM_STATIONS = len(station_ids)


In [ ]:
station_ids_set = set(station_ids)

def nearest_k_stations(K: int) -> dict[int, list[int]]:
    """Compute the K nearest stations to all stations, including itself.
    
    The returned dict is a mapping from (transformed) stationID to a list containing the 
    (transformed) station IDs of the nearest K stations.
    """
    ids = []
    coordinates = []
    status_i = 0
    information_i = 0
    while status_i < len(station_status) and information_i < len(station_information):
        status_station_id = int(station_status[status_i]["station_id"])
        information_station_id = int(station_information[information_i]["station_id"])
        if status_station_id < information_station_id:
            status_i += 1
        elif information_station_id < status_station_id:
            information_i += 1
        else:
            if station_status[status_i]["station_id"] in station_ids_set:
                # the tranformed stationID is the stationID used internally
                # it is simply the index of the stationID in the original_station_ids list
                transformed_station_id = len(ids)
                ids.append(transformed_station_id)
                coordinates.append((station_information[information_i]["lat"], station_information[information_i]["lon"]))
            status_i += 1
            information_i += 1
    
    k_nearest = {}
    for i, id in enumerate(ids):
        coordinate = coordinates[i]
        distances = [(distance.euclidean(x, coordinate), j) for j, x in enumerate(coordinates)]
        closest = heapq.nsmallest(K + 1, distances)
        k_nearest[id] = [ids[j] for _, j in closest[1:]]
    return k_nearest


Preprocess the data in the data frame. Further preprocessing will be done later.

In [ ]:
# Remove station ids that are now removed from the endpoints, but were present when data collection started (1 station was problematic)
df = df[df["StationID"].isin(station_ids_set)]

# df['StationID'] = df['StationID'].apply(station_encoding)
# df['WeatherStatus'] = df['WeatherStatus'].apply(weather_encoding)

# turn categorical variables into a one-hot encoding
# df = pd.get_dummies(df, columns=["WeatherStatus", "StationID"], drop_first=True)
df = pd.get_dummies(df, columns=["WeatherStatus",], drop_first=True)
df = df.drop('StationID', axis=1)

# Extract year, month, day of the year, hours, and minutes
df['year'] = df['Time'].dt.year
df['month'] = df['Time'].dt.month
df['dayoftheyear'] = df['Time'].dt.dayofyear
df['hours'] = df['Time'].dt.hour + (df['Time'].dt.minute / 60)
df['weekday'] = df['Time'].dt.weekday 
df = df.drop('Time', axis=1)


# TODO: consider normalizing other quantities later

# CHECK: do the boolean columns cause issues in training




In [ ]:
# TODO: check why this is different
# df.head(820)
# df.iloc[820]["StationID"]

In [ ]:
print(len(df))

8175265


Split data into train test validation

In [ ]:
# Constants for splitting
TRAIN_RATIO = 0.70
VALIDATION_RATIO = 0.15
TEST_RATIO = 1 - TRAIN_RATIO - VALIDATION_RATIO

# Ensure the ratios sum to 1
assert TRAIN_RATIO + VALIDATION_RATIO + TEST_RATIO == 1.0

# Calculate the indices for the splits
n = len(df) // NUM_STATIONS
train_end = floor(TRAIN_RATIO * n) * NUM_STATIONS  # First half for training
val_end = floor((TRAIN_RATIO + VALIDATION_RATIO) * n) * NUM_STATIONS  # Next quarter for validation

# Perform the split
df_train = df.iloc[:train_end]
df_val = df.iloc[train_end:val_end]
df_test = df.iloc[val_end:]

assert df_train.shape[0] + df_val.shape[0] + df_test.shape[0] == df.shape[0]
assert df_train.shape[0] % NUM_STATIONS == 0
assert df_val.shape[0] % NUM_STATIONS == 0 
assert df_test.shape[0] % NUM_STATIONS == 0 
# df_train, df_val, and df_test are your resulting splits



In [ ]:
print(df_train.columns)
df_test.to_numpy()
df_train.to_numpy()

Index(['NumBikes', 'NumDocks', 'Latitude', 'Longitude', 'Temperature',
       'WeatherStatus_701', 'WeatherStatus_741', 'WeatherStatus_800',
       'WeatherStatus_801', 'WeatherStatus_802', 'WeatherStatus_803',
       'WeatherStatus_804', 'year', 'month', 'dayoftheyear', 'hours',
       'weekday'],
      dtype='object')


array([[28, 12, 43.639832, ..., 207, 0.9, 3],
       [1, 13, 43.66496415990742, ..., 207, 0.9, 3],
       [0, 17, 43.667333, ..., 207, 0.9, 3],
       ...,
       [6, 30, 43.75404854399297, ..., 212, 13.8, 1],
       [2, 13, 43.76507271818177, ..., 212, 13.8, 1],
       [0, 19, 43.77121894873629, ..., 212, 13.8, 1]], dtype=object)

Create the torch Dataset to train the model

In [ ]:
class BikeShareDataset(Dataset):
    def __init__(self, data: pd.DataFrame, lookback: int, lookback_size: int, horizon: int, k: int) -> None:
        """
        Args:
            data (pd.DataFrame): DataFrame containing time series data.
            lookback (int): Number of time steps to look back.
            lookback_size (int): Length of time to jump over each lookback
            horizon (int): Number of time steps to predict.
            k (int):: Number of nearest stations information to consider in predictions.
        """
        self.FLAT_NUM_FEATURES = data.shape[1] * (k + 1)
        self.data = data.to_numpy()
        self.lookback = lookback
        self.lookback_size = lookback_size
        self.horizon = horizon
        self.k = k
        self.k_nearest_dict = nearest_k_stations(k)
        self.k_nearest_differences = self._compute_k_nearest_differences()

    def _compute_k_nearest_differences(self) -> dict[int, list[int]]:
        """Return the differences between the indices of neighbours in the pandas dataframe."""
        k_nearest_differences = defaultdict(list)
        for station in self.k_nearest_dict:
            for near_station in self.k_nearest_dict[station]:
                diff = near_station - station
                k_nearest_differences[station].append(diff)
            
        return k_nearest_differences
    
    def __len__(self) -> int:
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        padded_length = len(self.data) - (NUM_STATIONS * (self.horizon + FIRST_MINUTES_SKIPPED))
        padded_length = max(padded_length, 0)
        return padded_length
            
    def __getitem__(self, idx: int):
        FIRST_MINUTES_SKIPPED = self.lookback * self.lookback_size
        start_i = NUM_STATIONS * FIRST_MINUTES_SKIPPED
        idx = idx + start_i     
        current_station = idx % NUM_STATIONS
        x = np.ndarray((self.lookback, self.k + 1, self.data.shape[1]))
        for steps_behind in range(self.lookback):
            for k in range(self.k + 1):
                if k == 0:
                    x[steps_behind][k] = self.data[idx - (steps_behind * self.lookback_size * NUM_STATIONS)]
                else: 
                    x[steps_behind][k] = self.data[idx - (steps_behind * self.lookback_size * NUM_STATIONS) + self.k_nearest_differences[current_station][k - 1]]
        x = x.reshape(self.lookback, self.FLAT_NUM_FEATURES)

        y = np.ndarray((self.horizon))
        for minutes_ahead in range(1, self.horizon + 1):
            y[minutes_ahead - 1] = self.data[idx + (NUM_STATIONS * minutes_ahead)][0]  # index 0 is NumBikes
        return x, y
        

In [ ]:
lookback = 6
lookback_size = 5
horizon = 120
k = 5
train_data = BikeShareDataset(df_train, lookback, lookback_size, horizon, k)
validation_data = BikeShareDataset(df_val, lookback, lookback_size, horizon, k)
test_data = BikeShareDataset(df_test, lookback, lookback_size, horizon, k)

batch_size = 124
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

Test whether the dataset's __get__item__ works correctly

In [ ]:
# x = np.asarray(
#     [
#         [[1, 2],
#          [3, 4],
#          [1, 2]],

#         [[5, 6],
#          [7, 8],
#          [5, 6]],
#     ]
# )
# N, M, K = x.shape
# print(x.shape)
# flattened_x = x.reshape(N, M * K)
# print(flattened_x.shape)

# print(flattened_x)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cpu device


In [ ]:
FLAT_NUM_FEATURES = df.shape[1] * (k + 1)

class BikeShareLSTM(nn.Module):
    def __init__(self, hidden_size: int, horizon: int, lookback: int, num_layers: int):
        super().__init__()
        self.lookback = lookback
        self.flatten = nn.Flatten()
        self.lstm = nn.LSTM(input_size=FLAT_NUM_FEATURES, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True
        )
        self.hidden_size = hidden_size
        self.linear = nn.Linear((hidden_size * self.lookback), horizon)

    def forward(self, x):
        batch_size, sequence_length, features = x.shape
        x, _ = self.lstm(x)
        # x is of shape (batch_size, lookbacks, 50)
        x = x.reshape(batch_size, sequence_length * self.hidden_size)
        x = self.linear(x)
        return x

class BikeShareNN(nn.Module):
    def __init__(self, hidden_size: int, horizon: int, lookback: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.lookback = lookback
        self.horizon = horizon
        self.hidden_size = hidden_size
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(FLAT_NUM_FEATURES * lookback, self.hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, self.horizon),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [ ]:
HIDDEN_SIZE = 50
NUM_LAYERS = 5
# Uncomment for LSTM
#model = BikeShareLSTM(HIDDEN_SIZE, horizon, lookback, NUM_LAYERS).to(device)
#print(model)

model = BikeShareNN(HIDDEN_SIZE, horizon, lookback)  



In [ ]:
# loss_fn = nn.L1Loss(reduction="mean")
# batch = torch.tensor([[1, 2, 3], [1, 2, 3]], dtype=torch.float32)
# expected = torch.tensor([[1, 2, 3], [1, 2, 2]], dtype=torch.float32)

# sum(abs(batch - expected))

X, y = next(iter(train_dataloader))
print(y)

tensor([[ 6.,  6.,  6.,  ...,  9.,  9., 10.],
        [ 4.,  4.,  4.,  ...,  4.,  4.,  4.],
        [21., 21., 21.,  ...,  7.,  7.,  7.],
        ...,
        [14., 14., 14.,  ..., 11., 11., 11.],
        [ 9.,  9.,  9.,  ...,  9.,  9.,  9.],
        [ 4.,  4.,  4.,  ...,  7.,  7.,  8.]])


In [ ]:

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        assert pred.shape == y.shape
        total_loss_by_time = torch.zeros((horizon), dtype=float)
        loss = loss_fn(pred, y)
        total_loss_by_time += torch.sum(torch.abs(pred - y), dim=0)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            average_loss_by_time = total_loss_by_time / (batch_size)
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print(f"average loss by time {average_loss_by_time}")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss =  0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")



In [ ]:
learning_rate = 1e-2
epochs = 5

loss_fn = nn.L1Loss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(validation_dataloader, model, loss_fn)
    
print("Done!")

Epoch 1
-------------------------------
loss: 34.610062  [  124/5599865]
average loss by time tensor([ 36.4311,  80.6226,  64.8122,  33.9491,  52.1231,   8.4828,   5.9761,
         46.7278,  26.2456,   5.0962,  60.1098,  10.5333,  52.7787,   8.4992,
         63.3831,   8.4662,   6.6622,  16.1559,  37.8567,  12.9910,  34.6022,
         50.6888,  64.1698,  54.4755,  54.0087,  31.0969,   6.1398,   5.3943,
         43.6340,  21.5384,  40.4547,  16.5137,  56.0799,  12.3257,  19.1296,
         49.1351,  28.0119,  70.6817,   4.9868,  66.8319,  21.3097,  47.9546,
         57.8292,  49.2479,  56.2210,   4.9165,  17.6049,  38.3029,  32.6056,
         12.2303,   7.4100,  28.1315,  12.3335,   4.9472,  54.9073,  84.1805,
         21.8329,  14.8523,   5.5130,   5.8737,  52.3988, 128.7854,  13.9914,
         14.7053,  32.7729,  28.7151,  63.7839,  17.0054,  22.1223,  42.2768,
         13.4712,  34.4911,   4.9969,  26.4793,  34.2034,  14.0521, 112.3674,
         20.7027,  21.8439,  47.7120,  39.9807, 

KeyboardInterrupt: 